In [3]:
import cv2
from matplotlib import pyplot as plt
import glob
import os
import pandas as pd
import numpy as np
from shutil import copyfile
from sklearn.decomposition import PCA

# Functions for all methods.

In [4]:
def method1(path):
    img = cv2.imread(path,0)
    img = resizeImg(img)
    th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                cv2.THRESH_BINARY,11,2)
    edges = cv2.Canny(th2,100,200)
    return (edges//255).sum()

In [5]:
def method2(path):
    img = cv2.imread(path,0)
    img = resizeImg(img)
    kernel = np.array([[0,1,0],[1,1,1],[0,1,0]])
    dst = cv2.filter2D(img,-1,kernel)
    return ((dst-255)//127).sum()

In [6]:
def method3(path):
    img = cv2.imread(path,0)
    img = resizeImg(img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)
    cl1 = cl1//125
    kernel = np.array([[1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0],
                       [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0],
                       [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0],
                       [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0]]
                     )
    dst = cv2.filter2D(cl1,-1,kernel)
    kernel = np.array([[0,1,0],[1,1,1],[0,1,0]])
    dst = cv2.filter2D(dst,-1,kernel)
    edges = cv2.Canny(dst,100,200)
    n = edges.sum()//10000
    return n*n


In [7]:
def method4(path):
    img = cv2.imread(path,0)
    img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    try:
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                        cv2.THRESH_BINARY,11,2)
        faces_pca = PCA(n_components=2)
        faces_pca.fit(img)
        img_new = faces_pca.components_[0].reshape(16,16)+faces_pca.components_[1].reshape(16,16)
    except:
        return 0
    return img_new.sum()

In [8]:
def getcenterkernelized(path):
    img = cv2.imread(path,0)
    kernel = np.array([[0,1,0],
                       [0,1,0],
                       [0,1,0],
                       [0,1,0]])
    th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                    cv2.THRESH_BINARY,11,2)
    st = cv2.filter2D(img,-1,kernel)-255
    a,l = st.shape
    st = st[int(a*.1):int(a*.7),int(l*.1):int(l*.8)]
    return (st==255).sum()

In [9]:
def getColored(path):
    img = cv2.imread(path)
    img = resizeImg(img)
    if (img[:, :, 0]//127).sum()==(img[:, :, 1]//127).sum() and (img[:, :, 1]//127).sum()==(img[:, :, 2]//127).sum():
        return 0
    else:
        return 1

In [10]:
def sumblack(path):
    img = cv2.imread(path,0)
    img = resizeImg(img)
    return (img<=10).sum()

In [11]:
def resizeImg(img):
    resized = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
    return resized

# Below we must apply our methods to images.

In [26]:
function = method4 #choose the method you want to apply.
function_used = str(function).split(" ")[1]

In [14]:
mypath = 'images/'

### Applying to train data.

In [16]:
df = pd.read_csv("train.csv")
for k in range(0,len(df.fileName)):
    path = mypath+df.fileName.loc[k]
    df.iloc[k,4] = function(path)
df.to_csv(function_used+"_train.csv")

### Applying to test data.

In [212]:
df2 = pd.read_csv("sample_submission.csv")
for k in range(0,len(df2.fileName)):
    path = mypath+df2.fileName.loc[k]
    df2.iloc[k,1] = function(path)
df2.to_csv(function_used+"_test.csv")
  